# 載入基本程式庫

In [ ]:
import cv2
import numpy as np
import time, datetime
import requests, re, json

def process_image(img): #image binary
    img = cv2.imdecode(np.frombuffer(img, np.uint8), cv2.IMREAD_COLOR)
    size = img.shape
    width, height = 0,0
    if size[0]>size[1]: #size[0] to 1024
        width = 1024
        height = int(size[1]*(width/size[0]))
    else: #size[1] to 1024
        height = 1024
        width = int(size[0]*(height/size[1]))
    image = cv2.resize(img, (width,height))
    image = cv2.imencode(".jpg", image)[1].tobytes()
    return image

def observe(img, image_name):
    session = requests.Session()
    image = process_image(img)
    files={"image":(image_name, image)}
    payload = {"observed_on": get_time()} #使用現在日期時間
    req = session.post('https://api.inaturalist.org/v1/computervision/score_image', data=payload, headers=headers, files=files)
    result = req.content.decode(encoding="utf-8")
    result = json.loads(result)
    #print(result)
    spcies = []
    for i in result['results']: 
        try:
            spcies.append(i['taxon']['preferred_common_name'])
        except Exception as e:
            spcies.append(i['taxon']['name'])
        #print(spcies[-1])
    try:
        data = {"物種分類":result['common_ancestor']['taxon']["preferred_common_name"], "準確率":"%.2f"%result["common_ancestor"]["score"], "可能物種":spcies}
    except:
        data = {"準確率":"--", "物種分類":"無法辨識至屬，請嘗試裁減圖片，或者重新為生物照張相片","可能物種":spcies}
    #datas = json.dumps(data)
    return data

def start(url):
    r = requests.get(url, stream=True)
    if r.status_code==200:
      print(observe(r.raw.read(), "圖片1"))
    else:
      print("圖片抓取失敗，請確認圖片網址")

get_time = lambda: datetime.date.today().strftime("%Y/%m/%d")+" "+time.strftime("%H:%M:%S")

# 登錄您的iNaturalist帳號

In [ ]:
####登錄帳號####
account = input("請輸入帳號(please input account):")
password = input("請輸入密碼(please input password):")
r = requests.get("https://www.inaturalist.org/")
token = re.search('<meta name="csrf-token" content="(?P<token>[^"]+)"', r.text)["token"]
cookies = '_inaturalist_session:'+r.cookies.get_dict()['_inaturalist_session']
session = requests.Session()
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
           "Origin": "https://www.inaturalist.org",
           "Referer": "https://www.inaturalist.org/users/sign_in",}
payload = {"utf8": "✓",
           "authenticity_token": token,
           "user[email]": account,
           "user[password]": password,
           "user[remember_me]": "0",
           "user[remember_me]": "1"}
#req = session.post('https://api.inaturalist.org/v1/computervision/score_image', data=payload, headers=headers, files=files)
req = session.post("https://www.inaturalist.org/session", data=payload, headers=headers, cookies=r.cookies)
result = req.content.decode(encoding="utf-8")
Authorization = re.search('<meta name="inaturalist-api-token" content="(?P<token>[^"]+)"', result)["token"]
print("Authorization:",Authorization)
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
          "Authorization": Authorization,
           "Origin": "https://www.inaturalist.org",
           "Referer": "https://www.inaturalist.org/",
           "X-Via": "inaturalistjs"}

# 輸入圖片網址並取得觀察結果

In [ ]:
url = input("請輸入圖片網址: ")
start(url)